# 用RNN做文本生成
举个小小的例子，来看看LSTM是怎么玩的  
我们这里不再用char级别，我们用word级别来做。  
第一步，一样，先导入各种库

In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from gensim.models.word2vec import Word2Vec

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/jinfan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

接下来，我们把文本读入

In [3]:
raw_text = ''
for file in os.listdir("./input/"):
    if file.endswith(".txt"):
        raw_text += open("./input/"+file, errors='ignore').read() + '\n\n'
# raw_text = open('../input/Winston_Churchil.txt').read()
raw_text = raw_text.lower()
sentensor = nltk.data.load('tokenizers/punkt/english.pickle')     
sents = sentensor.tokenize(raw_text)
corpus = []
for sen in sents:
    corpus.append(nltk.word_tokenize(sen))

print(len(corpus))
print(corpus[:3])

91007
[['the', 'project', 'gutenberg', 'ebook', ',', 'oysters', 'and', 'fish', ',', 'by', 'thomas', 'j', '.'], ['(', 'thomas', 'jefferson', ')', 'murrey', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.'], ['you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www.gutenberg.org', '.']]


好，w2v乱炖：

In [4]:
w2v_model = Word2Vec(corpus, size=128, window=5, min_count=5, workers=4)

可以了

In [5]:
w2v_model['office']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.19270892,  0.12162855, -0.14810276, -0.01111651,  0.23072407,
       -0.00512885, -0.22223479,  0.00355991,  0.75731725,  0.2982118 ,
       -0.19129482,  0.03900472,  0.38711336,  0.22451423, -0.43874365,
        0.2865509 , -0.17099372,  0.12289249,  0.7095651 , -0.23809838,
       -0.12687337, -0.05366936, -0.23201181, -0.06480336, -0.6539293 ,
       -0.36103758,  0.58957523, -0.0852538 , -0.16164061,  0.0413104 ,
       -0.4863821 ,  0.8605279 , -0.24927779,  0.165787  ,  0.41577283,
        0.41274425, -0.7770883 ,  0.2604721 , -0.42613626,  0.10138069,
        0.3256224 , -0.11975811,  0.7702248 , -0.4822458 , -0.32087472,
       -0.33407477, -0.29361582, -0.2618962 ,  0.18204357,  0.11883137,
        0.30217448,  0.1596854 ,  0.40029874, -0.5480654 , -0.6952363 ,
        0.78702265,  0.09458437, -0.02461374,  0.07881169,  0.48614752,
       -0.11118302, -0.09023389, -0.21484718, -0.02474138, -0.18988189,
       -0.2324448 , -0.12519315, -0.5420816 , -0.08502176,  0.23

接下来，其实我们还是以之前的方式来处理我们的training data，把源数据变成一个长长的x，好让LSTM学会predict下一个单词：

In [6]:
raw_input = [item for sublist in corpus for item in sublist]
len(raw_input)

2163807

In [7]:
raw_input[12]

'.'

In [9]:
text_stream = []
vocab = w2v_model.wv.vocab
for word in raw_input:
    if word in vocab:
        text_stream.append(word)
len(text_stream)

2111782

我们这里的文本预测就是，给了前面的单词以后，下一个单词是谁?  
比如，hello from the other, 给出 side  
## 构造训练测试集
我们需要把我们的raw text变成可以用来训练的x,y:  
x 是前置字母们 y 是后一个字母

In [10]:
seq_length = 10
x = []
y = []
for i in range(0, len(text_stream) - seq_length):

    given = text_stream[i:i + seq_length]
    predict = text_stream[i + seq_length]
    x.append(np.array([w2v_model[word] for word in given]))
    y.append(w2v_model[predict])
    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


我们可以看看我们做好的数据集的长相：

In [11]:
print(x[10])
print(y[10])

[[ 0.29256192  0.06595486  0.02557228 ...  0.01049521 -0.31776536
   0.27186963]
 [ 0.0057021   0.01761983  0.01972884 ...  0.01804212 -0.00952614
   0.06724231]
 [-0.27348143  0.0164412   0.1176875  ...  0.456112   -0.98872805
   0.96937704]
 ...
 [ 0.03812757  0.685519   -1.5037822  ... -0.5998845   0.764545
  -0.3681679 ]
 [ 0.21473177  0.00915707 -0.31077555 ...  0.01241322  0.17599805
  -0.4713305 ]
 [-0.05145789 -0.20518498  1.4385833  ... -1.5857631  -1.2260756
  -1.7917078 ]]
[-2.057292    1.4928908  -0.57667494  0.38965368 -0.45786396 -0.4384504
  0.72420835  0.9672309  -1.983221    0.2218328  -0.35341883 -0.72874343
  0.9389885   0.45287448  1.3768233   0.68907756 -1.9638562   1.0806258
 -0.18083096  0.5511084   0.63933957  0.6183507  -0.47244754  0.7616638
 -1.1673354   1.108943    0.38035122 -0.1658669   0.8285052   0.39267507
 -0.47407213  0.556484    1.2145134   0.09423742  0.04607335 -0.42886657
  0.94338363 -1.0058573   0.491691   -0.01756302 -1.1016032  -0.6154638
  0.

In [12]:
print(len(x))
print(len(y))
print(len(x[12]))
print(len(x[12][0]))
print(len(y[12]))

2111772
2111772
10
128
128


In [13]:
x = np.reshape(x, (-1, seq_length, 128))
y = np.reshape(y, (-1,128))

接下来我们做两件事：  
我们已经有了一个input的数字表达（w2v），我们要把它变成LSTM需要的数组格式： [样本数，时间步伐，特征]  
第二，对于output，我们直接用128维的输出  
## 模型建造
LSTM模型构建

In [14]:
model = Sequential()
model.add(LSTM(256, dropout_W=0.2, dropout_U=0.2, input_shape=(seq_length, 128)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, input_shape=(10, 128), dropout=0.2, recurrent_dropout=0.2)`
  


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 跑模型

In [ ]:
model.fit(x, y, nb_epoch=50, batch_size=4096)

Instructions for updating:
Use tf.cast instead.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/50
2111772/2111772 [==============================] - 1593s 754us/step - loss: 0.7404
Epoch 2/50
 757760/2111772 [=========>....................] - ETA: 18:12 - loss: 0.7252

我们来写个程序，看看我们训练出来的LSTM的效果：

In [ ]:
def predict_next(input_array):
    x = np.reshape(input_array, (-1,seq_length,128))
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    raw_input = raw_input.lower()
    input_stream = nltk.word_tokenize(raw_input)
    res = []
    for word in input_stream[(len(input_stream)-seq_length):]:
        res.append(w2v_model[word])
    return res

def y_to_word(y):
    word = w2v_model.most_similar(positive=y, topn=1)
    return word

In [ ]:
def generate_article(init, rounds=30):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_word(predict_next(string_to_index(in_string)))
        in_string += ' ' + n[0][0]
    return in_string

In [ ]:
init = 'Language Models allow us to measure how likely a sentence is, which is an important for Machine'
article = generate_article(init)
print(article)